# How to save/load your model
1. load the weights
2. load the graph

In [1]:
import numpy as np
import tensorflow as tf
from pprint import pprint

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 4], name='Input')
y = tf.placeholder(tf.float32, shape=[None, 1], name='Output')

h1 = tf.layers.dense(x, units=10, activation=tf.nn.relu, name='hidden1')
y_pred = tf.layers.dense(h1, units=1, name='output')

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_pred), name='loss')

train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
init = tf.global_variables_initializer()

saver = tf.train.Saver()

pprint(tf.global_variables())

[<tf.Variable 'hidden1/kernel:0' shape=(4, 10) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'output/kernel:0' shape=(10, 1) dtype=float32_ref>,
 <tf.Variable 'output/bias:0' shape=(1,) dtype=float32_ref>]


In [3]:
sess = tf.Session()
sess.run(init)

In [4]:
X_train = np.tile(np.array([1, 2, 3]).reshape(-1, 1), 4)
y_train = np.array([1, 1, 0]).reshape(-1, 1)

print('X:')
print(X_train)
print('y:')
print(y_train)

X:
[[1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]]
y:
[[1]
 [1]
 [0]]


In [5]:
print('before training:')
print('predict: ', sess.run(tf.nn.sigmoid(y_pred), feed_dict={x: X_train}))
print('loss: ', sess.run(loss, feed_dict={x: X_train, y:y_train}))

for i in range(1000):
    sess.run(train_op, feed_dict={x: X_train, y:y_train})

print('')
print('after training:')
print('predict: ', sess.run(tf.nn.sigmoid(y_pred), feed_dict={x: X_train}))
print('loss: ', sess.run(loss, feed_dict={x: X_train, y:y_train}))

before training:
predict:  [[0.55424577]
 [0.6072295 ]
 [0.65780365]]
loss:  0.72045535

after training:
predict:  [[0.99979264]
 [0.9793102 ]
 [0.01401627]]
loss:  0.011743218


In [6]:
saver.save(sess, "./save_model/checkpoint_weight.ckpt")  # save the model
sess.close()

## 1. load the weights
#### You have to rewrite the graph.

In [7]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 4], name='Input')
y = tf.placeholder(tf.float32, shape=[None, 1], name='Output')

h1 = tf.layers.dense(x, units=10, activation=tf.nn.relu, name='hidden1')
y_pred = tf.layers.dense(h1, units=1, name='output')

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_pred), name='loss')

train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

saver = tf.train.Saver()

In [8]:
sess = tf.Session()
saver.restore(sess, "./save_model/checkpoint_weight.ckpt")

INFO:tensorflow:Restoring parameters from ./save_model/checkpoint_weight.ckpt


In [9]:
X_train = np.tile(np.array([1, 2, 3]).reshape(-1, 1), 4)
y_train = np.array([1, 1, 0]).reshape(-1, 1)

print('predict: ', sess.run(tf.nn.sigmoid(y_pred), feed_dict={x: X_train}))
print('loss: ', sess.run(loss, feed_dict={x: X_train, y:y_train}))

predict:  [[0.99979264]
 [0.9793102 ]
 [0.01401627]]
loss:  0.011743218


In [10]:
sess.close()

## 2. load the graph
#### You have to keep tensors' names.

In [11]:
sess = tf.Session()

loader = tf.train.import_meta_graph("./save_model/checkpoint_weight.ckpt" + '.meta')
loader.restore(sess, "./save_model/checkpoint_weight.ckpt")

pprint(tf.global_variables())

INFO:tensorflow:Restoring parameters from ./save_model/checkpoint_weight.ckpt
[<tf.Variable 'hidden1/kernel:0' shape=(4, 10) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'output/kernel:0' shape=(10, 1) dtype=float32_ref>,
 <tf.Variable 'output/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'hidden1/kernel:0' shape=(4, 10) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'output/kernel:0' shape=(10, 1) dtype=float32_ref>,
 <tf.Variable 'output/bias:0' shape=(1,) dtype=float32_ref>]


In [12]:
X_train = np.tile(np.array([1, 2, 3]).reshape(-1, 1), 4)
y_train = np.array([1, 1, 0]).reshape(-1, 1)

x = sess.graph.get_tensor_by_name('Input:0')
y = sess.graph.get_tensor_by_name('Output:0')
y_pred = sess.graph.get_tensor_by_name('output_1/BiasAdd:0')
loss = sess.graph.get_tensor_by_name('loss:0')

print('predict: ', sess.run(tf.nn.sigmoid(y_pred), feed_dict={x: X_train}))
print('loss: ', sess.run(loss, feed_dict={x: X_train, y:y_train}))

predict:  [[0.99979264]
 [0.9793102 ]
 [0.01401627]]
loss:  0.011743218


In [ ]:
sess.close()